# Lab-06-1 Softmax Classifier
## 소프트맥스 분류기 실행

In [1]:
import tensorflow as tf
import numpy as np
## dataset
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
##원핫 인코딩의 결과
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format (넘파이, 실수로 변환)
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)
nb_classes = 3 #class의 개수입니다.


#계수와 상수
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight') ## x 원소 4개
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]
## ver1에서는 
#W=tfe.Variable(tf.random_normal([4, nb_classes]), name="weight")
#b=tfe.Variable(tf.random_normal([nb_classes]), name='bias')

# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print("here")
print(sample_db)

#cost 엔트로피
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1) #-y*log(hypothesis) 의 합
    cost_mean = tf.reduce_mean(cost) #경사면을 타고 내려와서 감소시킴
    return cost_mean
#print(cost_fn(x_data, y_data))

#경사하강법
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
#print(grad_fn(x_data, y_data))


def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)##Gradient decent optimizer
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

here
[[8. 2. 1. 4.]]
Loss at epoch 1: 2.480121
Loss at epoch 100: 0.715940
Loss at epoch 200: 0.631831
Loss at epoch 300: 0.574215
Loss at epoch 400: 0.524550
Loss at epoch 500: 0.477868
Loss at epoch 600: 0.432423
Loss at epoch 700: 0.387459
Loss at epoch 800: 0.342674
Loss at epoch 900: 0.298410
Loss at epoch 1000: 0.258429
Loss at epoch 1100: 0.238360
Loss at epoch 1200: 0.226894
Loss at epoch 1300: 0.216447
Loss at epoch 1400: 0.206877
Loss at epoch 1500: 0.198079
Loss at epoch 1600: 0.189965
Loss at epoch 1700: 0.182459
Loss at epoch 1800: 0.175498
Loss at epoch 1900: 0.169024
Loss at epoch 2000: 0.162991


In [4]:
sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a=hypothesis(sample_data)

print(a)
print(tf.argmax(a,1))

tf.Tensor([[5.1197840e-04 8.1583895e-02 9.1790408e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [6]:
a=hypothesis(x_data)

print(a)
##제일 확률이 높은것을 선택
print(tf.argmax(a, 1))
##실제 y값과 비교
print(tf.argmax(y_data, 1))
##정상적으로 수행되었음

tf.Tensor(
[[1.4303945e-06 1.2640662e-03 9.9873453e-01]
 [5.1197869e-04 8.1583902e-02 9.1790408e-01]
 [3.3300174e-07 1.6542630e-01 8.3457333e-01]
 [7.4715190e-06 8.4981847e-01 1.5017402e-01]
 [2.6801020e-01 7.1996796e-01 1.2021818e-02]
 [1.4159882e-01 8.5837191e-01 2.9242756e-05]
 [7.3066771e-01 2.6929209e-01 4.0235289e-05]
 [9.2462069e-01 7.5378664e-02 6.0974958e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [8]:
class softmax_classifier(tf.keras.Model):
    def __init__(self,nb_classes):
        super(softmax_classifier, self).__init__()
        self.W = tf.Variable(tf.random.normal((4,nb_classes)),name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)),name ='bias')
        
    def softmax_regression(self,X):
        return tf.nn.softmax(tf.matmul(X,self.W)+self.b)
    
    def cost_fn(self, X,Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.math.log(logits), axis=1))
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
                
model = softmax_classifier(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.031656
Loss at epoch 500: 0.512542
Loss at epoch 1000: 0.286504
Loss at epoch 1500: 0.203797
Loss at epoch 2000: 0.166570
